In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Create example data
# 1.1 Setup a single Bracewell

In [ ]:
# combiner matrix for a single Bracewell array
combiner = np.array(((1, -1), (1, 1))) / np.sqrt(2)

# collector positions
baseline = 15  # in meter

# rotation angles over observation
rotation_angle = np.linspace(0., 2*np.pi, 1000)  # in rad

collector_positions_init = np.array(((-baseline/2, baseline/2), (0, 0)))

rotation_matrix = np.array(((np.cos(rotation_angle), -np.sin(rotation_angle)),
                            (np.sin(rotation_angle), np.cos(rotation_angle))))

collector_position = np.dot(np.swapaxes(rotation_matrix, -1, 0), collector_positions_init)

# observing wavelength
wl_bin = 10e-6  # in meter

# collector area
scaled_area = 1  # in meter^2

## 1.2 Compute planet signal

In [ ]:
# source position
separation_arcsec = 0.1  # in arcsec
separation_rad = separation_arcsec / 3600 / 180 * np.pi  # in rad

# rotation angles over observation
rotation_angle_planet = 0. # in rad

# source position angle vectors in radians
source_position = np.array((separation_rad * np.cos(rotation_angle_planet), 
                            separation_rad * np.sin(rotation_angle_planet)))

# Calculate the phase term
phase_term = 1j * 2 * np.pi / wl_bin

# Compute the dot products for all t in one go using np.tensordot
dot_product_x = np.tensordot(collector_position[:, 0, :], source_position, 
                             axes=([1], [0]))
dot_product_y = np.tensordot(collector_position[:, 1, :], source_position, 
                             axes=([1], [0]))

# Calculate the phasors
phasor_x = scaled_area * np.exp(phase_term * dot_product_x)
phasor_y = scaled_area * np.exp(phase_term * dot_product_y)

# Combine the results
input_phasor = np.array([phasor_x, phasor_y])

planet_signal = np.abs(np.array([np.dot(combiner, input_phasor[:, t]) for t in range(input_phasor.shape[1])]))**2

## 1.3 Add random noise and plot the signal

In [ ]:
star_signal = np.random.normal([np.mean(planet_signal)*10, np.mean(planet_signal)*1000], [np.sqrt(np.mean(planet_signal)*10), np.sqrt(np.mean(planet_signal)*1000)], planet_signal.shape)

signal = planet_signal + star_signal

In [ ]:
plt.plot(signal[:, 1])
plt.plot(signal[:, 0])
plt.ylabel('Signal in a.u.')
plt.xlabel('Time in a.u.')
plt.show()

# 2. Initialize a nifits object
## 2.1 Initialize the extension classes

In [ ]:
from nifits.io.oifits import NI_CATM
ni_catm = NI_CATM(Mcn=combiner[np.newaxis, :, :])

In [ ]:
from nifits.io.oifits import NI_FOV
ni_fov = NI_FOV(mode='radial',
                char_scale=1.)

In [ ]:
from nifits.io.oifits import OI_TARGET
oi_target = OI_TARGET(target=['Test Target'], 
                      raep0=14.3, 
                      decep0=-60.4)

In [ ]:
from nifits.io.oifits import NI_MOD
ni_mod = NI_MOD(app_index=[0, 1],
                target_id=[0, 0], 
                time=3600., 
                mjd=60396.041666,
                int_time=1000.,
                mod_phas=np.ones((1, signal.shape[0]), dtype=np.complex128),
                app_xy=collector_position, 
                arrcol=1.,
                fov_index=0)

In [ ]:
import datetime
# print current time and date as string
now = str(datetime.datetime.now())

In [ ]:
str(now)

In [ ]:
header_dict = {'SIMPLE': True,
               'BITPIX': -32,
               'NAXIS': 0,
                'EXTEND': True,
               'ORIGIN': 'NIFITS Consortium',
               'DATE': str(datetime.datetime.now()),
               'DATE-OBS': '2021-01-01T00:00:00',
               'CONTENT': 'NIFITS',
                'TELESCOP': 'Test Telescope',
                'INSTRUME': 'Test Instrument',
                'OBJECT': 'Test Object',
                'OBSERVER': 'Test Observer',
               'INSMODE': 'Test Instrument Mode',
               }

## 2.2 Initialize the OIFITS object

In [ ]:
from nifits.io.oifits import oifits

In [ ]:
oifits_obj = oifits(header_dict=header_dict,
                    catm=ni_catm,
                    fov=ni_mod)